<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Movie-recommendation" data-toc-modified-id="Movie-recommendation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Movie recommendation</a></span><ul class="toc-item"><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dataset</a></span></li><li><span><a href="#Evaluation-Protocol" data-toc-modified-id="Evaluation-Protocol-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Evaluation Protocol</a></span></li><li><span><a href="#Models" data-toc-modified-id="Models-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Models</a></span><ul class="toc-item"><li><span><a href="#ALS" data-toc-modified-id="ALS-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-collaborative-filtering.html#explicit-vs-implicit-feedback" target="_blank">ALS</a></a></span></li><li><span><a href="#Ваша-формулировка" data-toc-modified-id="Ваша-формулировка-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Ваша формулировка</a></span></li></ul></li><li><span><a href="#Evaluation-Results" data-toc-modified-id="Evaluation-Results-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Evaluation Results</a></span></li></ul></li></ul></div>

# Movie recommendation

Ваша задача - рекомендация фильмов для пользователей


In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.memory", "12g") \
    .config("spark.task.cpus", "8") \
    .config("spark.executor.cores", "8") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

## Dataset 

`MovieLens-25M`

In [2]:
DATA_PATH = '/workspace/data/ml-25m'

RATINGS_PATH = os.path.join(DATA_PATH, 'ratings.csv')
MOVIES_PATH = os.path.join(DATA_PATH, 'movies.csv')
TAGS_PATH = os.path.join(DATA_PATH, 'tags.csv')

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *


ratings_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + RATINGS_PATH)

In [4]:
ratings_df.limit(5).toPandas()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


## Evaluation Protocol

Так как мы хотим оценивать качество разных алгоритмов рекомендаций, то в первую очередь нам нужно определить
* Как разбить данные на `Train`/`Validation`/`Test`;
* Какие метрики использовать для оценки качества.

Выбрал способ разбиения №1, чтобы точно не заглядывать в будущее.  

Также explicit feedback (т.е. не делать бинарным рейтинг — например понравился фильм или нет — rating > 3), так как есть уже собранный датасет с explicit, что круто

Добавил опцию, чтобы оставить только тех пользователей, которые встречаются во всех частях

In [5]:
from pyspark.sql.window import Window

In [6]:
def split_by_col(df, split_col, parts_fractions, common_users=False, user_col=None):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    # from homework № 3
    df_with_rank = df \
      .withColumn('rank', F.percent_rank().over(Window.orderBy(split_col))) # (sorting inside)
    
    parts = []    
    cumsum = 0
    
    for part_ratio in parts_fractions:
        part = df_with_rank \
                    .filter((cumsum <= df_with_rank['rank'])  \
                              & (df_with_rank['rank'] < cumsum + part_ratio)) \
                    .drop('rank') \
                    .persist()
        parts.append(part)
        
        cumsum += part_ratio
    
    if common_users:
        assert user_col, "Specify column of user to leave only common"
        
        users = [part.select(F.col(user_col)).distinct() for part in parts]
        
        common_users = users[0]
        for users_part in users[1:]:
            common_users = common_users.join(users_part, on=user_col)
            
        parts = [part.join(common_users, on=user_col) for part in parts]       
    
    return parts

In [7]:
train_df, val_df, test_df = split_by_col(ratings_df, 'timestamp', [0.8, 0.1, 0.1], common_users=True, user_col='userId')
train_df, val_df, test_df = train_df.cache(), val_df.cache(), test_df.cache()

In [9]:
N = ratings_df.count()
train_df.count() / N, val_df.count() / N, test_df.count() / N

Для оценивания будем использовать Average Treatment Effect и [RankingMetrics](https://spark.apache.org/docs/2.3.0/api/python/pyspark.mllib.html#pyspark.mllib.evaluation.RankingMetrics):
* MAP
* NDCG 
* Precision at k
А также можно подойти как к задаче регрессии: а именно разность предсказания и настоящей оценки пользователя

In [10]:
def get_ate(groups, control_name) -> pd.DataFrame:
    """Get Average Treatment Effect
    groups - dictionary where keys - names of models, values - dicts of pairs <metric_name>, <metric_value>
    control_name - name of baseline model
    
    return pd.DataFrame (rows corresponds to metrics, cols corresponds to models and ATE with respect to control)
    """
    metrics = pd.DataFrame.from_records(groups, columns=groups.keys())
    metrics.index.name = 'metric'
    return metrics.subtract(metrics[control_name], axis='index') * 100

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.mllib.evaluation import RankingMetrics

In [12]:
def test_model(df, model):
    # regression: evaluate the model by computing the RMSE
    predictions = model.transform(df)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                    predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)
    regression_metrics = {'rmse': rmse}
    
    # ranking
    top_Ns = [1, 5, 10]
    
    users = df.select('userId').distinct()
    recommendations = model.recommendForUserSubset(users, numItems=max(top_Ns))
    
    true_items = df \
        .groupby('userId') \
        .agg(F.collect_set('movieId').alias('true_movies'))
    
    pred_and_true = true_items \
        .join(recommendations, 'userId') \
        .select('recommendations', 'true_movies') \
        .rdd \
        .map(lambda row: (list(map(lambda x: x.movieId, row[0])), row[1]))
    
    ranking_metrics = RankingMetrics(pred_and_true)    
    ranking_metrics_values = {
        **{"Precision@{}".format(N): ranking_metrics.precisionAt(N) for N in top_Ns},
        **{"NDCG@{}".format(N): ranking_metrics.ndcgAt(N) for N in top_Ns},
        "MAP@{}".format(max(top_Ns)) : ranking_metrics.meanAveragePrecision
    }
    
    return {**regression_metrics, **ranking_metrics_values}

## Models

Теперь мы можем перейти к формулировке задачи в терминах машинного обучения.

Одна из формулировок, к которой мы сведем нашу задачу - **Matrix Completetion**. Данную задачу будем решать с помощью `ALS`

### [ALS](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html#explicit-vs-implicit-feedback)

In [13]:
from pyspark.ml.recommendation import ALS

In [14]:
ALS_static_params = {
    'implicitPrefs': False,
    'userCol': 'userId',
    'itemCol': 'movieId',
    'ratingCol': 'rating',
    'coldStartStrategy': 'drop',
}

In [15]:
als = ALS(**ALS_static_params)

In [14]:
%%time
als_baseline_model = als.fit(train_df)

CPU times: user 28 ms, sys: 12.1 ms, total: 40.1 ms
Wall time: 2min 48s


In [15]:
als_baseline_metrics = test_model(test_df, als_baseline_model)

In [16]:
als_baseline_metrics

{'MAP@10': 5.559622687812465e-06,
 'NDCG@1': 0.0,
 'NDCG@10': 0.0004756600202195305,
 'NDCG@5': 0.00036512865098158216,
 'Precision@1': 0.0,
 'Precision@10': 0.0005679919812896758,
 'Precision@5': 0.00046775810223855684,
 'rmse': 0.9421837158466277}

In [17]:
from collections import OrderedDict

In [18]:
all_metrics = OrderedDict()

In [19]:
all_metrics['als_baseline'] = als_baseline_metrics

Покажите для выбранных вами фильмов топ-20 наиболее похожих фильмов

In [143]:
movies_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + MOVIES_PATH)

In [144]:
from scipy.spatial.distance import cosine 

@F.udf(returnType=FloatType())
def cosine_similarity(x, y):
    return float(1 - cosine(x, y))

In [145]:
def most_similar_movies(movie_id, model_features, top_n=10):
    movie_features = model_features \
                    .filter(F.col('id') == movie_id) \
                    .select(F.col('id').alias('movie_id'), F.col('features').alias('movie_features'))
    
    similar = model_features \
                    .crossJoin(movie_features) \
                    .withColumn('sim', cosine_similarity('movie_features', 'features')) \
                    .select(F.col('id').alias('movieId'), 'sim') \
                    .sort(F.col('sim').desc()) \
                    .limit(top_n)
    
    return similar.join(movies_df, on='movieId').sort(F.col('sim').desc()) 

In [148]:
most_similar_movies(1, als_baseline_model.itemFactors).toPandas()

,movieId,sim,title,genres
0,1,1.000000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,3114,0.996000,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
2,2355,0.992900,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy
3,4886,0.977806,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy
4,80862,0.975387,Catfish (2010),Documentary|Mystery
5,8961,0.972890,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy
6,6377,0.972354,Finding Nemo (2003),Adventure|Animation|Children|Comedy
7,2294,0.969500,Antz (1998),Adventure|Animation|Children|Comedy|Fantasy
8,78737,0.969368,Captain from Castile (1947),Adventure|Drama
9,2078,0.969146,"Jungle Book, The (1967)",Animation|Children|Comedy|Musical


История игрушек 2 похожа на Историю игрушек 1, что вполне оправданно  

0, 1, 2, 3, 5, 6 - фильмы Pixar

In [149]:
most_similar_movies(296, als_baseline_model.itemFactors).toPandas()

,movieId,sim,title,genres
0,296,1.000000,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1089,0.990733,Reservoir Dogs (1992),Crime|Mystery|Thriller
2,67168,0.978129,Dance of the Dead (2008),Adventure|Comedy|Horror
3,778,0.978029,Trainspotting (1996),Comedy|Crime|Drama
4,555,0.975035,True Romance (1993),Crime|Thriller
5,4251,0.970117,Chopper (2000),Drama|Thriller
6,1222,0.969624,Full Metal Jacket (1987),Drama|War
7,223,0.968471,Clerks (1994),Comedy
8,26574,0.968295,Ginger and Fred (Ginger e Fred) (1986),Comedy|Drama
9,6874,0.965939,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller


3 фильма от Тарантино (True Romance - сценарист)

In [150]:
most_similar_movies(260, als_baseline_model.itemFactors).toPandas()

,movieId,sim,title,genres
0,260,1.000000,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
1,1196,0.995534,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
2,1210,0.984139,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
3,1198,0.983977,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure
4,1374,0.976134,Star Trek II: The Wrath of Khan (1982),Action|Adventure|Sci-Fi|Thriller
5,1291,0.974766,Indiana Jones and the Last Crusade (1989),Action|Adventure
6,77966,0.971665,Love the Beast (2009),Documentary
7,8771,0.966122,Sherlock Holmes: Terror by Night (1946),Crime|Mystery|Thriller
8,73981,0.963986,Man Hunt (1941),Crime|Drama|Thriller
9,2716,0.962633,Ghostbusters (a.k.a. Ghost Busters) (1984),Action|Comedy|Sci-Fi


В топе звездные войны

In [151]:
most_similar_movies(58559, als_baseline_model.itemFactors).toPandas()

,movieId,sim,title,genres
0,58559,1.000000,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
1,33794,0.993886,Batman Begins (2005),Action|Crime|IMAX
2,79132,0.988752,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX
3,49272,0.988206,Casino Royale (2006),Action|Adventure|Thriller
4,48780,0.984723,"Prestige, The (2006)",Drama|Mystery|Sci-Fi|Thriller
5,54997,0.981426,3:10 to Yuma (2007),Action|Crime|Drama|Western
6,76251,0.981426,Kick-Ass (2010),Action|Comedy
7,77833,0.980662,Armadillo (2010),Documentary|Drama
8,79333,0.980321,Watch Out for the Automobile (Beregis avtomobi...,Comedy|Crime|Romance
9,71817,0.980078,Beer Wars (2009),Documentary


0, 1, 2, 4 - фильмы Нолана

### Ваша формулировка

На лекции было еще несколько ML формулировок задачи рекомендаций. Выберете одну из них и реализуйте метод.

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

## Evaluation Results

Сравните реализованные методы с помощью выбранных метрик. Не забывайте про оптимизацию гиперпараметров.

In [20]:
! pip3.5 install hyperopt

In [16]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

should_cast_to_int = ['rank', 'maxIter', 'numUserBlocks', 'numItemBlocks']

def cast_to_int(space):
    for param in should_cast_to_int:
        space[param] = int(space[param])
    return space

def objective(space):
    space = cast_to_int(space)
    print("\nSpace: ", space)
    als = ALS(**space)
    fitted_model = als.fit(train_df)
    print("Done training")
    metrics = test_model(val_df, fitted_model)    
    print(metrics)

    return {'loss': -metrics['MAP@10'], 'status': STATUS_OK }

In [17]:
als_space = {
    # Optimize
    'rank': hp.quniform('rank', 2, 50, 4),
    'maxIter': hp.quniform('maxIter', 2, 30, 4),
    'regParam': hp.loguniform('regParam', -4, 0),
    'numUserBlocks': hp.quniform('numUserBlocks', 2, 30, 4),
    'numItemBlocks': hp.quniform('numItemBlocks', 2, 30, 4),
    
    **ALS_static_params
}

In [ ]:
trials = Trials()
best = fmin(fn=objective,
            space=als_space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

                                                      
Space: 
{'coldStartStrategy': 'drop', 'userCol': 'userId', 'implicitPrefs': False, 'rank': 12, 'numUserBlocks': 12, 'ratingCol': 'rating', 'regParam': 0.029980628141101903, 'maxIter': 8, 'itemCol': 'movieId', 'numItemBlocks': 16}
{'NDCG@1': 0.0006682258603407954, 'rmse': 0.859194142209738, 'NDCG@10': 0.0010494608114461977, 'Precision@5': 0.0009355162044771135, 'MAP@10': 1.5458243788489057e-05, 'Precision@10': 0.001169395255596392, 'Precision@1': 0.0006682258603407954, 'NDCG@5': 0.000836594019627714}
                                                                                       
Space: 
{'coldStartStrategy': 'drop', 'userCol': 'userId', 'implicitPrefs': False, 'rank': 16, 'numUserBlocks': 8, 'ratingCol': 'rating', 'regParam': 0.04498241642290061, 'maxIter': 12, 'itemCol': 'movieId', 'numItemBlocks': 20}
  5%|▌         | 1/20 [04:10<1:19:22, 250.67s/trial, best loss: 1.5458243788489057e-05]

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################